[DNN tutorial](https://pythonprogramming.net/train-test-tensorflow-deep-learning-tutorial/?completed=/preprocessing-tensorflow-deep-learning-tutorial/)

[markdown syntax](https://www.markdownguide.org/basic-syntax/)

# TO DO:
- write a function to do all the image processing
- make notes on what all the tensorflow function I use do
- figure out how to do one-hot encoding

### Folder Structure

- Annotations
- Images
- ImageSets
- random_attack
    - all-random-bomb
    - all-random-flower
    - all-random-ysq
- targeted_attack
    - stop-speedlimit-bomb
    - stop-speedlimit-flower
    - stop-speedlimit-ysq

# Loading Data

In [23]:
import os
import random
import skimage.data
import skimage.transform
from skimage import io
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Allow image embeding in notebook
%matplotlib inline

categories = {'warning':0, 'speedlimit':1, 'stop':2,
             0:'warning', 1:'speedlimt', 2:'stop'}

def load_data(data_dir, ann_dir):
    #returns a tuple of the relevant images and the relevant labels
    labels, images = [], []
    x1, x2, y1, y2 = 0, 0, 0, 0
    count = 0 #REMOVE WHEN MORE COMPUTING AVAILABLE
    with open(data_dir) as imset:
        for cur_im in imset:
            if count > 1000: #REMOVE WHEN MORE COMPUTING AVAILABLE ##################
                break
            if cur_im.endswith("\n"):
                cur_im = cur_im[:-1] 
            with open(os.path.join(ann_dir, cur_im + ".txt")) as annotation:
                for anno in annotation:
                    label,x1,y1,x2,y2,clean = anno.split(',')
                    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                    if "clean" in data_dir:
                        image = skimage.data.imread(os.path.join("Images", cur_im+".png"))
                    elif "ysq" in data_dir:
                        if os.path.exists(os.path.join("targeted_attack","stop-speedlimit-ysq",cur_im+".png")):
                            image = skimage.data.imread(os.path.join("targeted_attack","stop-speedlimit-ysq",cur_im+".png"))
                            label = 'speedlimit'
                        else:
                            image = skimage.data.imread(os.path.join("Images", cur_im+".png"))
                    labels.append(categories[label])
                    max_h, max_w = image.shape[0], image.shape[1]
                    image = skimage.util.crop(image,((y1, max_h - y2),(x1,max_w - x2),(0,0)), copy=False)
                    images.append(image)
                    count += 1 #REMOVE WHEN MORE COMPUTING AVAILABLE
        return images, labels

train_data_dir = os.path.join("ImageSets", "test_ysq.txt")
test_data_dir = os.path.join("ImageSets", "test_targ_ysg_backdoor.txt")
anno_dir = "Annotations"

print("Sanity Check:")
images, labels = load_data(train_data_dir, anno_dir)
print("Unique Labels: {0}\nTotal Images: {1}".format(len(set(labels)), len(images)))

Sanity Check:
Unique Labels: 3
Total Images: 262


In [24]:
def display_images_and_labels(images, labels):
    """Display the first image of each label."""
    unique_labels = set(labels)
    plt.figure(figsize=(15, 15))
    i = 1
    for label in unique_labels:
        # Pick the first image for each label.
        image = images[labels.index(label)]
        plt.subplot(8, 8, i)  # A grid of 8 rows x 8 columns
        plt.axis('off')
        plt.title("{0} ({1})".format(label, labels.count(label)))
        i += 1
        _ = plt.imshow(image)
    plt.show()
    
def display_label_images(images, label):
    """Display images of a specific label."""
    limit = 24  # show a max of 24 images
    plt.figure(figsize=(15, 5))
    i = 1

    start = labels.index(label)
    end = start + labels.count(label)
    for image in images[start:end][:limit]:
        plt.subplot(3, 8, i)  # 3 rows, 8 per row
        plt.axis('off')
        i += 1
        plt.imshow(image)
    plt.show()

In [25]:
# Resizing image
images32 = [skimage.transform.resize(image, (32, 32), mode='constant')
                for image in images]
# display_images_and_labels(images32, labels)

# Creating DNN

In [26]:
n_nodes_hl1 = 500
n_nodes_hl2 = 500
n_nodes_hl3 = 500
n_classes = 3
batch_size = 100

size_dataset = len(images32)

# Flatten input from: [None, height, width, channels]
# To: [None, height * width * channels] == [None, 3072]
x = tf.placeholder('float', [None, 3072])
y = tf.placeholder('int64')

images32_flat = np.asarray(images32).flatten().reshape(size_dataset, 3072)

In [27]:
def neural_network_model(data):
    #keep adding layers until test error stops improving
    hidden_1_layer = {'weights':tf.Variable(tf.random_normal([3072, n_nodes_hl1])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl1]))}

    hidden_2_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl2]))}

    hidden_3_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl3]))}

    output_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
                    'biases':tf.Variable(tf.random_normal([n_classes]))}

    # bias is a value that is added to our sums, before being passed through the activation function
    # purpose of the bias here is mainly to handle for scenarios where all neurons fired a 0 into the layer
    # bias makes it possible that a neuron still fires out of that layer
    # a bias is as unique, and also needs to be optimized
    
    l1 = tf.add(tf.matmul(data,hidden_1_layer['weights']), hidden_1_layer['biases'])
    l1 = tf.nn.relu(l1)

    l2 = tf.add(tf.matmul(l1,hidden_2_layer['weights']), hidden_2_layer['biases'])
    l2 = tf.nn.relu(l2)

    l3 = tf.add(tf.matmul(l2,hidden_3_layer['weights']), hidden_3_layer['biases'])
    l3 = tf.nn.relu(l3)

    output = tf.matmul(l3,output_layer['weights']) + output_layer['biases']

    return output

def next_batch(num, data, labels):
    '''
    Return a total of 'num' random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]
    return np.asarray(data_shuffle), np.asarray(labels_shuffle)


In [28]:
def train_neural_network(x):
    prediction = neural_network_model(x)
    cost = tf.reduce_mean( tf.nn.sparse_softmax_cross_entropy_with_logits(logits=prediction, labels=tf.squeeze(y)) )
      
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    hm_epochs = 10
    #epochs = cycles of feed forward and back prop
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        for epoch in range(hm_epochs):
            epoch_loss = 0
            for _ in range(int(size_dataset/batch_size)):
                epoch_x, epoch_y = next_batch(batch_size, images32_flat, labels)
                _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
                epoch_loss += c

            print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)
          
        correct = tf.equal(tf.argmax(prediction, 1), y)        
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        
        test_images, test_labels = load_data(train_data_dir, anno_dir)
        test_images = [skimage.transform.resize(image, (32, 32), mode='constant') for image in test_images]
        test_images = np.asarray(test_images).flatten().reshape(size_dataset, 3072)
        
        print(accuracy.eval({x:test_images, y:test_labels}))
        
train_neural_network(x)

Epoch 0 completed out of 10 loss: 54359.03515625
Epoch 1 completed out of 10 loss: 61498.9609375
Epoch 2 completed out of 10 loss: 30361.2890625
Epoch 3 completed out of 10 loss: 35070.73046875
Epoch 4 completed out of 10 loss: 18923.6572265625
Epoch 5 completed out of 10 loss: 16151.81787109375
Epoch 6 completed out of 10 loss: 16207.84814453125
Epoch 7 completed out of 10 loss: 10961.255859375
Epoch 8 completed out of 10 loss: 8703.977294921875
Epoch 9 completed out of 10 loss: 8091.245361328125
0.7557252


## Identifying Backdoor

[paper](http://people.cs.uchicago.edu/~ravenben/publications/pdf/backdoor-sp19.pdf)

In [ ]:
# define a genericform of trigger injection:
# A(x,m, ∆) = x′
# x′_(i,j,c) = (1 − m_(i,j)) · x_(i,j,c) + m_(i,j) · ∆_(i,j,c)

# A(·) represents the function that applies a trigger to the original image, x. 
# ∆ is the trigger pattern, which is a 3D matrix of pixel color intensities with the same 
#       dimension of the input image (height, width, and color channel). 
# m is a 2D matrix called the mask, deciding how much the trigger can overwrite the original image.
# Values in the mask range from 0 to 1 (1 copmletely overwrites the image)

# The optimization has two objectives. 
# For a given target label to be analyzed (y_t), the first objective is to find a
# trigger (m, ∆) that would misclassify clean images into y_t.
# The second objective is to find a “concise” trigger, meaning a trigger that only modifies a 
# limited portion of the image.
# We measure the magnitude of the trigger by the L1 norm of the mask m. 
# Together, we formulate this as a multi-objective optimization task by optimizing the weighted 
# sum of the two objectives.

# min_(m,∆) ℓ(y_t, f(A(x,m, ∆))) + λ · |m|
# for x ∈ X

# f(·) is the DNN’s prediction function. 
# ℓ(·) is the loss function measuring the error in classification, which is cross
#      entropy in our experiment. 
# λ is the weight for the second objective.

# Then of these idetified backdoors for each find the Smallest one.
# i.e. the one with the smallest L_1 norm

#### NOTES:
- a **deep** neural network has more than 2 layers
- **logit** is a function that maps probabilities \[0, 1\] to \[-inf, +inf\].
- **softmax** is a function that maps \[-inf, +inf\] to \[0, 1\] similar as Sigmoid. 
    - softmax also normalizes the sum of the values(output vector) to be 1.
- **tensorflow "with logit"** means that you are applying a softmax function to logit numbers to normalize it. 
    - the input_vector/logit is not normalized and can scale from \[-inf, inf\].
- tensorflow **graph** vs **session**:
    - a **graph** defines the computation. It doesn’t compute anything, it doesn’t hold any values, it just defines the operations that you specified in your code.
    - a **session** allows to execute graphs or part of graphs. It allocates resources (on one or more machines) for that and holds the actual values of intermediate results and variables.
    - [source](https://danijar.com/what-is-a-tensorflow-session/)
- **one_hot encoding** should be used with categorical data
    - mapping labels to integers may cause unwanted side effect (1<2<3 etc.)
    - [stack exchange](https://datascience.stackexchange.com/questions/30215/what-is-one-hot-encoding-in-tensorflow?rq=1)